In [1]:
!pip install pyspark

    100% |████████████████████████████████| 204.7MB 265kB/s eta 0:00:01
    100% |████████████████████████████████| 204kB 31.2MB/s ta 0:00:01
  Running setup.py install for pyspark ... done


In [32]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.classification import DecisionTreeClassificationModel


from pyspark.sql import SparkSession

In [11]:
spark = SparkSession.builder \
     .master("local") \
     .appName("Word Count") \
     .getOrCreate()

# Prepare training data from a list of (label, features) tuples.
training = spark.createDataFrame([
    (1.0, Vectors.dense([0.0, 1.1, 0.1])),
    (0.0, Vectors.dense([2.0, 1.0, -1.0])),
    (0.0, Vectors.dense([2.0, 1.3, 1.0])),
    (1.0, Vectors.dense([0.0, 1.2, -0.5]))], ["label", "features"])


In [24]:
dtc = DecisionTreeClassifier(maxDepth=3)
dtc_model = dtc.fit(training)

In [40]:
dtc_model.parent.extractParamMap

AttributeError: 'DecisionTreeClassificationModel' object has no attribute 'parent'

In [33]:
def _get_root_node(tree: DecisionTreeClassificationModel):
    return tree._call_java('rootNode')

In [34]:
_get_root_node(dtc_model)

JavaObject id=o781

In [20]:
# Prepare test data
test = spark.createDataFrame([
    (1.0, Vectors.dense([-1.0, 1.5, 1.3])),
    (0.0, Vectors.dense([3.0, 2.0, -0.1])),
    (1.0, Vectors.dense([0.0, 2.2, -1.5]))], ["label", "features"])

# Make predictions on test data using the Transformer.transform() method.
# LogisticRegression.transform will only use the 'features' column.
# Note that model2.transform() outputs a "myProbability" column instead of the usual
# 'probability' column since we renamed the lr.probabilityCol parameter previously.
prediction = model1.transform(test)
result = prediction.select("features", "label", "prediction") \
    .collect()

for row in result:
    print("features=%s, label=%s -> , prediction=%s"
          % (row.features, row.label, row.prediction))

features=[-1.0,1.5,1.3], label=1.0 -> , prediction=1.0
features=[3.0,2.0,-0.1], label=0.0 -> , prediction=0.0
features=[0.0,2.2,-1.5], label=1.0 -> , prediction=1.0
